<a href="https://colab.research.google.com/github/pukung375/CE888/blob/master/Lab5/Recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
from IPython.display import Image
np.set_printoptions(precision = 3)

#Load the data from ``jester-data-1.csv''

In [0]:
df = pd.read_csv('https://raw.githubusercontent.com/albanda/CE888/master/lab5-recommender/jester-data-1.csv', header=None)
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
0,74,-7.82,8.79,-9.66,-8.16,-7.52,-8.50,-9.85,4.17,-8.98,-4.76,-8.50,-6.75,-7.18,8.45,-7.18,-7.52,-7.43,-9.81,-9.85,-9.85,-9.37,1.50,-4.37,-9.81,-8.50,1.12,7.82,2.86,9.13,-7.43,2.14,-4.08,-9.08,7.82,5.05,4.95,-9.17,-8.40,-8.40,...,8.59,3.59,-6.84,-9.03,2.82,-1.36,-9.08,8.30,5.68,-4.81,99.00,99.00,99.00,99.00,99.00,99.00,99.00,-9.42,99.00,99.00,99.00,-7.72,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,2.82,99.00,99.00,99.00,99.00,99.00,-5.63,99.00,99.00,99.00
1,100,4.08,-0.29,6.36,4.37,-2.38,-9.66,-0.73,-5.34,8.88,9.22,6.75,8.64,4.42,7.43,4.56,-0.97,4.66,-0.68,3.30,-1.21,0.87,8.64,8.35,9.17,0.05,7.57,4.71,0.87,-0.39,6.99,6.50,-0.92,7.14,9.03,-1.80,0.73,7.09,3.40,-0.87,...,-6.70,-3.35,-9.03,4.47,4.08,-3.83,8.74,1.12,0.78,7.52,-5.00,2.77,8.30,7.77,7.33,6.21,7.72,8.98,8.64,8.20,3.93,4.85,4.85,6.07,8.98,4.51,-0.05,3.69,4.56,0.58,2.82,-4.95,-0.29,7.86,-0.19,-2.14,3.06,0.34,-4.32,1.07
2,49,99.00,99.00,99.00,99.00,9.03,9.27,9.03,9.27,99.00,99.00,7.33,7.57,9.37,6.17,-6.36,-6.89,-7.86,9.03,9.03,9.03,7.28,99.00,8.25,99.00,99.00,7.48,7.28,7.28,8.93,99.00,6.17,7.28,99.00,99.00,8.98,7.33,99.00,6.17,9.08,...,6.46,7.28,99.00,99.00,7.04,7.28,99.00,7.28,8.25,99.00,99.00,99.00,99.00,99.00,99.00,8.93,99.00,99.00,99.00,9.08,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,9.03,99.00,99.00,99.00,9.08,99.00,99.00,99.00,99.00,99.00,99.00
3,48,99.00,8.35,99.00,99.00,1.80,8.16,-2.82,6.21,99.00,1.84,7.33,6.60,6.31,8.11,-7.23,-6.65,1.17,-6.60,-3.64,-2.09,5.34,99.00,99.00,99.00,99.00,2.91,3.93,6.75,6.60,99.00,6.65,-6.12,99.00,7.57,6.21,6.65,99.00,-8.30,7.18,...,0.00,-3.69,99.00,99.00,7.82,0.24,99.00,7.28,-2.33,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,0.63,99.00,99.00,-2.33,99.00,99.00,99.00,99.00,99.00,0.53,99.00,99.00,99.00,99.00,99.00,99.00
4,91,8.50,4.61,-4.17,-5.39,1.36,1.60,7.04,4.61,-0.44,5.73,8.25,6.84,-3.93,7.23,-2.33,-9.66,2.72,-1.36,2.57,4.51,8.20,6.12,8.30,-1.26,7.77,1.89,-1.17,5.68,8.45,4.61,8.06,-9.47,7.28,5.68,2.48,3.20,-1.26,6.80,4.51,...,7.38,6.17,4.71,-2.28,7.38,4.56,7.14,4.22,3.01,3.83,99.00,99.00,99.00,99.00,99.00,99.00,4.13,99.00,99.00,99.00,5.24,5.92,0.87,7.28,3.93,-0.63,6.31,4.71,2.82,2.96,5.19,5.58,4.27,5.19,5.73,1.55,3.11,6.55,1.80,1.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24978,100,0.44,7.43,9.08,2.33,3.20,6.75,-8.79,-0.53,-8.74,7.23,-0.53,5.63,-7.14,-4.08,-3.50,-8.20,-3.98,-9.22,-0.15,-6.46,5.63,-0.92,-2.91,-4.17,2.82,3.40,8.64,6.84,6.80,-0.87,7.38,-3.50,8.88,7.43,5.39,2.23,-0.68,3.40,-0.58,...,8.59,3.45,0.87,9.27,-4.66,5.73,-0.49,8.35,1.94,5.00,-9.66,8.98,8.98,-9.81,9.13,9.08,9.08,3.98,0.73,9.03,8.98,9.22,8.93,9.13,9.27,-1.99,-9.95,-9.90,9.13,8.83,8.83,-1.21,9.22,-6.70,8.45,9.03,6.55,8.69,8.79,7.43
24979,91,9.13,-8.16,8.59,9.08,0.87,-8.93,-3.50,5.78,-8.11,4.90,8.88,-8.69,-7.48,-8.83,-1.75,6.60,3.54,1.50,7.67,-0.44,9.22,8.74,9.03,9.08,8.93,3.74,3.20,-9.17,-8.98,8.79,-7.67,-3.06,9.13,8.40,-0.63,-7.18,0.58,8.88,9.27,...,2.77,8.11,-7.96,8.93,-0.87,-5.87,8.88,-1.12,-8.74,8.74,99.00,99.00,99.00,99.00,99.00,4.90,99.00,99.00,99.00,99.00,-0.29,0.92,-0.78,0.15,-0.10,0.00,-0.19,-0.87,-1.36,-0.58,-1.17,-5.73,-1.46,0.24,9.22,-8.20,-7.23,-8.59,9.13,8.45
24980,39,99.00,99.00,99.00,99.00,-7.77,99.00,6.70,-6.75,99.00,99.00,99.00,99.00,-6.46,-1.65,-6.80,-6.41,-6.99,7.23,6.75,-6.99,6.55,99.00,99.00,99.00,99.00,0.49,-0.53,-6.94,-0.49,99.00,6.46,-0.53,99.00,99.00,-7.86,-0.34,99.00,-6.94,99.00,...,0.49,-0.24,99.00,99.00,-3.11,-6.65,99.00,-0.58,6.31,99.00,99.00,-7.86,99.00,99.00,

In [0]:
arr = df.copy() 
arr = arr.drop(arr.columns[0], axis=1)

In [0]:
arr = df.values
print(arr.shape)

(24983, 101)


In [0]:
rated = np.where(arr!=99)
rated

(array([    0,     0,     0, ..., 24982, 24982, 24982]),
 array([ 0,  1,  2, ..., 70, 71, 87]))

In [0]:
rated = np.where(arr!=99)
print(len(rated[0]), rated[1].shape)

1835357 (1835357,)


In [0]:
def replace(orig, percentage=0.1):
  """
  Replaces 'percentage'% of the original values in 'orig' with 99's
  :param orig: original data array
  :param percentage: percentage of values to replace (0<percentage<1)
  """
  new_data = orig.copy()
  rated = np.where(arr!=99)
  n_rated = len(rated[0])
  idx = np.random.choice(n_rated, size=int(percentage*n_rated), replace=False)
  new_data[rated[0][idx], rated[1][idx]] = 99
  return new_data, (rated[0][idx], rated[1][idx])

In [0]:
new_arr, idx = replace(arr, 0.1)

In [0]:
arr[idx[0][0], idx[1][0]]

1.75

In [0]:
new_arr[idx[0][0], idx[1][0]]

99.0

In [0]:
n_latent_factors = 2

user_ratings = df.values
# Initialise as random values
latent_user_preferences = np.random.random((user_ratings.shape[0], n_latent_factors))
latent_item_features = np.random.random((user_ratings.shape[1], n_latent_factors))

In [0]:
def predict_rating(user_id, item_id):
    """ Predict a rating given a user_id and an item_id.
    """
    user_preference = latent_user_preferences[user_id]
    item_preference = latent_item_features[item_id]
    return user_preference.dot(item_preference)


def train(user_id, item_id, rating, alpha=0.0001):
    
    #print item_id
    prediction_rating = predict_rating(user_id, item_id)
    err =  prediction_rating - rating
    #print err
    user_pref_values = latent_user_preferences[user_id][:]
    latent_user_preferences[user_id] -= alpha * err * latent_item_features[item_id]
    latent_item_features[item_id] -= alpha * err * user_pref_values
    return err
    

def sgd(iterations=1000, alpha=0.0001):
    """ Iterate over all users and all items and train for 
        a certain number of iterations
    """
    for iteration in range(iterations):
        error = []
        for user_id in range(latent_user_preferences.shape[0]):
            for item_id in range(latent_item_features.shape[0]):
                rating = user_ratings[user_id][item_id]
                if (rating!=99):
                    err = train(user_id, item_id, rating)
                    error.append(err)
        mse = (np.array(error) ** 2).mean()   
        if (iteration % 1) == 0:
            print(mse)

In [0]:
# caseA1: latent_factors=2, iterations=20, alpha=0.0001
sgd(iterations=20, alpha=0.0001)  # Note how the MSE decreases with the number of iterations

57.96549805499956
31.390232103846078
25.263775983887083
24.70128895288731
24.65517984415492
24.65001283801267
24.64866310238358
24.647781064152632
24.646963792889075
24.646104028562142
24.645105895328392
24.643832281615705
24.642069725016576
24.639477439986763
24.635504759856047
24.629257233751474
24.61928078861563
24.603217124525663
24.577262582550052
24.535342829291597


In [0]:
predictions = latent_user_preferences.dot(latent_item_features.T)
predictions

array([[ 7.501e+01,  6.349e-01,  8.426e-02, ...,  6.020e-01, -2.028e-01,
         9.650e-01],
       [ 9.636e+01,  1.096e+00,  3.584e-01, ...,  9.731e-01,  5.516e-02,
         1.460e+00],
       [ 5.464e+01,  4.888e-01,  8.478e-02, ...,  4.573e-01, -1.182e-01,
         7.237e-01],
       ...,
       [ 3.812e+01,  4.088e-01,  1.196e-01, ...,  3.672e-01, -6.166e-03,
         5.582e-01],
       [ 3.757e+01,  3.761e-01,  9.394e-02, ...,  3.429e-01, -3.630e-02,
         5.291e-01],
       [ 7.449e+01,  9.700e-01,  3.864e-01, ...,  8.396e-01,  1.807e-01,
         1.226e+00]])

In [0]:
values = [zip(user_ratings[i], predictions[i]) for i in range(predictions.shape[0])]
comparison_data = pd.DataFrame(values)
comparison_data.columns = df.columns

In [0]:
comparison_data
# For each data point, the number on the left is the original value from the dataset, the number on the right is the prediction

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
0,"(74.0, 75.00612934856188)","(-7.82, 0.6349391595475771)","(8.79, 0.08426032520999148)","(-9.66, 0.19963732886413085)","(-8.16, -1.4125798971433394)","(-7.52, 0.28893063706150623)","(-8.5, 1.2699333969705695)","(-9.85, -0.5349908658900054)","(4.17, -0.62284107581935)","(-8.98, -0.7548796872780339)","(-4.76, 0.8969110989179678)","(-8.5, 1.4182384738807463)","(-6.75, 1.127036219845066)","(-7.18, -1.7683906421613949)","(8.45, 1.0337863137294987)","(-7.18, -1.7905329934513883)","(-7.52, -3.0961821285441076)","(-7.43, -1.1295403034666558)","(-9.81, -0.7284247480727313)","(-9.85, -0.01530096564341363)","(-9.85, -1.0944601960886384)","(-9.37, 1.7854668821824915)","(1.5, 0.5099465503917571)","(-4.37, -0.2728104109564088)","(-9.81, -1.6644823405862623)","(-8.5, 0.11800388176991676)","(1.12, 0.9641597964120486)","(7.82, 2.9069533204322973)","(2.86, 1.2255597487976322)","(9.13, 2.650843417237708)","(-7.43, -0.6525560815661894)","(2.14, 1.9782134481677296)","(-4.08, 2.8469976527880574)","(-9.08, -1.4063966528446517)","(7.82, 0.6158227902103878)","(5.05, 2.651696857467743)","(4.95, 2.990348966267062)","(-9.17, -1.4443330778188155)","(-8.4, 0.8949177901376586)","(-8.4, 0.7414697228102347)",...,"(8.59, 2.1515355768827233)","(3.59, 2.6105584595348414)","(-6.84, -0.07619936429489638)","(-9.03, -0.8042725091911235)","(2.82, 2.044979635832833)","(-1.36, 2.209833098388726)","(-9.08, -1.0316544078792418)","(8.3, 2.329034179876734)","(5.68, 2.311527866924378)","(-4.81, 0.17316117359328068)","(99.0, -0.6667633415575049)","(99.0, 2.1990987989562165)","(99.0, 0.7952570084902391)","(99.0, -1.2491601886842825)","(99.0, -0.13395412978091395)","(99.0, 2.0190461942013953)","(99.0, 0.4842687796354831)","(-9.42, 1.3235757941559139)","(99.0, 0.14135991171310838)","(99.0, 0.9615241208846049)","(99.0, 1.460138280424198)","(-7.72, 0.6595308906150962)","(99.0, 1.6179857188619882)","(99.0, 0.4165506555726262)","(99.0, 0.6541621856695881)","(99.0, 0.26620477020513156)","(99.0, 1.4067507052014707)","(99.0, 1.5906690135863673)","(99.0, 2.667752920564097)","(99.0, 0.41762772982901863)","(2.82, 1.5736145666530603)","(99.0, 0.8719362659165635)","(99.0, 1.9583197208937158)","(99.0, 0.7984152561722353)","(99.0, 0.626873634079843)","(99.0, 1.1222572860794617)","(-5.63, 1.1256983204527529)","(99.0, 0.6019989941865428)","(99.0, -0.20284188307428253)","(99.0, 0.9650437293018728)"
1,"(100.0, 96.3570077960247)","(4.08, 1.0961193702347787)","(-0.29, 0.35838320723862727)","(6.36, 0.4224768388543642)","(4.37, -1.5043206751588505)","(-2.38, 0.5624588181326362)","(-9.66, 1.823640259149032)","(-0.73, -0.4189759388679811)","(-5.34, -0.6473111793768915)","(8.88, -0.6185393070128326)","(9.22, 1.5253556754724196)","(6.75, 2.16655104477441)","(8.64, 1.804688069775279)","(4.42, -2.149576927547476)","(7.43, 1.7320689696086191)","(4.56, -2.093810551830051)","(-0.97, -3.744421024608138)","(4.66, -1.2945245391030256)","(-0.68, -0.7677881045901899)","(3.3, 0.2366635876828555)","(-1.21, -1.1726903761028065)","(0.87, 2.713291547990915)","(8.64, 1.1186660814444465)","(8.35, 0.1957842447680187)","(9.17, -1.8288898244662737)","(0.05, 0.5229202997180028)","(7.57, 1.598582625954017)","(4.71, 3.9563501940834747)","(0.87, 1.858878758663287)","(-0.39, 3.7441176464634176)","(6.99, -0.48370093762004657)","(6.5, 2.781786862919353)","(-0.92, 3.927973054095307)","(7.14, -1.4538038386124035)","(9.03, 1.1498205942487045)","(-1.8, 3.769345582236323)","(0.73, 4.114265540285397)","(7.09, -1.476622301275144)","(3.4, 1.5933254541421986)","(-0.87, 1.3854774872035784)",...,"(-6.7, 3.075771860241795)","(-3.35, 3.70647040641401)","(-9.03, 0.2969366478559327)","(4.47, -0.6570984795831871)","(4.08, 2.9187122091219266)","(-3.83, 3.1687617867739393)","(8.74, -0.8946145481584769)","(1.12, 3.

In [0]:
# Calculate the performance (e.g., MSE) of the algorithm on the validation dataset
error = (predictions[idx[0][:],idx[1][:]]-arr[idx[0][:],idx[1][:]])**2
mseA1=sum(error)/len(error)
print("caseA1: ",mseA1)

caseA1:  24.506369752671773


Repeat the two points above changing hyper-parameters (i.e., learning rate, number of iterations of SVD, number of latent factors, etc.) as needed to get good results (you can create multiple validation sets if you want, and run a bootstrap!)

In [0]:
# caseA2: latent_factors=2, iterations=30, alpha=0.0001
n_latent_factors = 2
user_ratings = new_arr
np.random.seed(1)
latent_user_preferences = np.random.random((user_ratings.shape[0], n_latent_factors))
latent_item_features = np.random.random((user_ratings.shape[1], n_latent_factors))
sgd(iterations=15,alpha=0.0001)
predictions = latent_user_preferences.dot(latent_item_features.T)
error = (predictions[idx[0][:],idx[1][:]]-arr[idx[0][:],idx[1][:]])**2
mseA2=sum(error)/len(error)
print("caseA2: ",mseA2)

60.1321917780049
32.058313661646125
25.47985278182832
24.71338417720438
24.631563268206186
24.611355580330244
24.59776692956536
24.58546861405063
24.573802629304325
24.56261370350433
24.5518008269262
24.541248133244608
24.530798350228725
24.520224885536322
24.50919129658779
caseA2:  40.38410888844587


In [0]:
# caseA2: latent_factors=2, iterations=30, alpha=0.0001
n_latent_factors = 2
user_ratings = new_arr
np.random.seed(1)
latent_user_preferences = np.random.random((user_ratings.shape[0], n_latent_factors))
latent_item_features = np.random.random((user_ratings.shape[1], n_latent_factors))
sgd(iterations=30,alpha=0.0001)
predictions = latent_user_preferences.dot(latent_item_features.T)
error = (predictions[idx[0][:],idx[1][:]]-arr[idx[0][:],idx[1][:]])**2
mseA2=sum(error)/len(error)
print("caseA2: ",mseA2)

60.1321917780049
32.058313661646125
25.47985278182832
24.71338417720438
24.631563268206186
24.611355580330244
24.59776692956536
24.58546861405063
24.573802629304325
24.56261370350433
24.5518008269262
24.541248133244608
24.530798350228725
24.520224885536322
24.50919129658779
24.497189499577072
24.48344519343043
24.4667737141032
24.445362308386137
24.41644626618284
24.375839218132253
24.317280430108124
24.231594450592077
24.10576394079595
23.92226859066293
23.659520377216243
23.29486659069954
22.81186301064788
22.21188170738513
21.5252452352359
caseA2:  45.78733594543292


In [0]:
# caseA3: latent_factors=2, iterations=40, alpha=0.0001
n_latent_factors = 2
user_ratings = new_arr
np.random.seed(1)
latent_user_preferences = np.random.random((user_ratings.shape[0], n_latent_factors))
latent_item_features = np.random.random((user_ratings.shape[1], n_latent_factors))
sgd(iterations=40,alpha=0.0001)
predictions = latent_user_preferences.dot(latent_item_features.T)
error = (predictions[idx[0][:],idx[1][:]]-arr[idx[0][:],idx[1][:]])**2
mseA3=sum(error)/len(error)
print("caseA3: ",mseA3)

60.1321917780049
32.058313661646125
25.47985278182832
24.71338417720438
24.631563268206186
24.611355580330244
24.59776692956536
24.58546861405063
24.573802629304325
24.56261370350433
24.5518008269262
24.541248133244608
24.530798350228725
24.520224885536322
24.50919129658779
24.497189499577072
24.48344519343043
24.4667737141032
24.445362308386137
24.41644626618284
24.375839218132253
24.317280430108124
24.231594450592077
24.10576394079595
23.92226859066293
23.659520377216243
23.29486659069954
22.81186301064788
22.21188170738513
21.5252452352359
20.81163082651254
20.14246075417658
19.573449824491508
19.1272900081051
18.796702882633884
18.55922303553109
18.390347789005954
18.26983911263935
18.182918919817677
18.11935662736073
caseA3:  43.28096558593547


In [0]:
# caseB1: latent_factors=3, iterations=20, alpha=0.0001
n_latent_factors = 3
user_ratings = new_arr
np.random.seed(1)
latent_user_preferences = np.random.random((user_ratings.shape[0], n_latent_factors))
latent_item_features = np.random.random((user_ratings.shape[1], n_latent_factors))
sgd(iterations=40,alpha=0.0001)
predictions = latent_user_preferences.dot(latent_item_features.T)
error = (predictions[idx[0][:],idx[1][:]]-arr[idx[0][:],idx[1][:]])**2
mseB1=sum(error)/len(error)
print("caseB1: ",mseB1)

52.92388325845578
31.126941478712233
26.02575979848225
24.886093042101407
24.65621102983484
24.600517385075356
24.57793106430558
24.560914301704386
24.54312976924008
24.521910390078272
24.494654271619744
24.457748235711257
24.405841099619256
24.331027260362102
24.221897345159086
24.062659556814975
23.832933741448933
23.509445216093198
23.0714065246515
22.51076361914311
21.844910255210284
21.12324419142956
20.41698303193601
19.792942582789156
19.28899497287619
18.908682498835145
18.633338795367933
18.437534486998846
18.298437682894303
18.198739745406424
18.126277106683258
18.07274276490634
18.03248923447047
18.001651131298367
17.97755182838916
17.958311891089522
17.942591118217173
17.92941743312524
17.918072277852023
17.90801303042142
caseB1:  35.12305448970637


In [0]:
# caseB2: latent_factors=4, iterations=30, alpha=0.0001
n_latent_factors = 4
user_ratings = new_arr
np.random.seed(1)
latent_user_preferences = np.random.random((user_ratings.shape[0], n_latent_factors))
latent_item_features = np.random.random((user_ratings.shape[1], n_latent_factors))
sgd(iterations=40,alpha=0.0001)
predictions = latent_user_preferences.dot(latent_item_features.T)
error = (predictions[idx[0][:],idx[1][:]]-arr[idx[0][:],idx[1][:]])**2
mseB2=sum(error)/len(error)
print("caseB2: ",mseB2)

48.83429863619704
30.831657764786517
26.540468639631072
25.16204167424905
24.754502541891153
24.630085228829678
24.586605351394503
24.566260206216217
24.552301034677143
24.539473187510218
24.52560331334798
24.509104217185776
24.488087775881453
24.459851811287535
24.420362534613492
24.3635967125414
24.28070362503694
24.1590615541253
23.98154342593841
23.7267647703872
23.371734876056877
22.30635115637876
21.62185404889653
20.902592782435843
20.220409273142316
19.634076456949757
19.170050067784164
18.823616348617456
18.573217305132367
18.394182677160618
18.26570680282146
18.172416404044952
18.10358308601706
18.051857474526656
18.01220168496164
17.981129769225827
17.95619760563608
17.935665338397918
17.918273483498776
caseB2:  34.02795290249243


In [0]:
# caseB3: latent_factors=5, iterations=40, alpha=0.0001
n_latent_factors = 5
user_ratings = new_arr
np.random.seed(1)
latent_user_preferences = np.random.random((user_ratings.shape[0], n_latent_factors))
latent_item_features = np.random.random((user_ratings.shape[1], n_latent_factors))
sgd(iterations=40,alpha=0.0001)
predictions = latent_user_preferences.dot(latent_item_features.T)
error = (predictions[idx[0][:],idx[1][:]]-arr[idx[0][:],idx[1][:]])**2
mseB3=sum(error)/len(error)
print("caseB3: ",mseB3)

45.701115620230716
30.60445861964782
26.94855095195479
25.472973867642946
24.911886150018265
24.698450302401636
24.61349092274839
24.575441273637452
24.554114073597237
24.538019885244896
24.522272346647494
24.50406467867437
24.480847472228742
24.44940481641394
24.405148120721055
24.34136332369689
24.248337691753534
24.112470217371907
23.91576884593499
23.636641436265396
23.253478545132378
22.752515558298708
22.13936794353551
21.448580969132667
20.741203613508777
20.085486640883484
19.53117539395372
19.095983150948097
18.770813455598784
18.533935440015608
18.362481359931845
18.237642117475666
18.14553558163693
18.076368329186945
18.02332103230335
17.9816331568905
17.94794860862841
17.919869016993683
17.89565210743648
17.874007598923033
caseB3:  31.958280203107382
